***

## Multitape Turing Machines

>csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2020.
<br>
Burton Rosenberg.
<br>
<br>
Created: 24 March 2020
<br>last update: 4 April 2020

***


### Overview

There are many ways to vary the definition of the Turing Machine. It is important that no matter what that variation (within the bounds of reasonablity) the resulting machine cannot surpass the Turing Machine in power. That is, it is impossible that there is a set A that the variant machine can decide but the standard definition Turing Machine cannot (and the same for recognize, in place of decide). This is important because what we learn from Turing Machines should be about computation, in general. For this we have to check that the conclusions we draw from considering the standard definition Turing Machine would be the same as we can draw from _any_ computing device. (Including analog computers, GPU's, or Quantum Computers).

To prove that the Turing Machine can do whatever some other machine M can do, we create from the description of M a descrption T of a Turing Machine that does exactly what M does. One approach is _simulation_. Just as our Python runtime is simulating a Turing Machine, taking a Turing Machine description and walking through its calculation step by step, modeling the tape as a list, and the state function as a Python Dictionary, the stand defintion Turing Machine is programmed to follow step by step what the variant machine does. 

The two variants of interest are:
- multiple tapes and 
- nondeterminism. 

In this Notebook we consider a standard defintion Turing Machine simulation of a multi-tape Turing Machine.

### Multi-tape TM

A multi-tape TM has more than one tape, with an independent head on each tape; and the transitions take into consideration the symbols all tapes, and can write and act on each tape. For instance, a 2-tape TM would have transitions,

 $$
 (q_1, \sigma_1, \sigma_2 ) \longrightarrow (q_2, \sigma'_1, \rho_1, \sigma'_2, \rho_2)
 $$
 
This means that when the state is $q_1$, and tape 1 has symbol $\sigma_1$, and tape 2 has symbol $\sigma_2$, then,
1. transition into state $q_2$, 
2. and write on tape one the symbol $\sigma'_1$ 
3. and take action $\rho_1$ on tape one, 
4. and write on tape two the symbol $\sigma'_2$
5. and take action $\rho_2$ on tape two.

The simulate a multi-tape machine with just one tape, we use two ideas,
1. We combine all the tapes onto one tape. We will interleave the tapes onto one tape.
2. Our simulating machine keeps trace of the simulated computation using a _soft-state_, basically like our variable current\_state in the Python code.


**Interleaving the tapes: from many, one**

The complete description of the tape contents will show that the tape is actually a _data structure_. It will have the following segments, 
1. A soft-state area, 
2. followed by a staging-area,
3. followed by an interleaved-tapes area.
We first describe the interleaved-tapes area, for the example of two tapes into one.

The syntax for the interleaved area is,
Let tape one be the sequence, 

$$
\tau_{1,0}\,\tau_{1,1}\,\tau_{1,2}\,\tau_{1,4}\,\tau_{1,5}\, ...
$$

and tape two be the sequence, 

$$
\tau_{2,0}\,\tau_{2,1}\,\tau_{2,2}\,\tau_{2,4}\,\tau_{2,5}\, ...
$$

And let $\delta_{1,i}$ be the character blank except when $i$ is equal to the head position on tape 1, and then it is $1$; and likes $\delta_{2,i}$, except at the appropriate $i$ it is $2$.

Then the interleaved area is,

$$
 \delta_{1,0}\, \tau_{1,0} \,\delta_{2,0}\, \tau_{2,0}\, \delta_{1,1}\, \tau_{1,1}\, \delta_{2,1}\, \tau_{2,1} ...
$$

We assume that the symbols 1 and 2 are not tape symbols for the original tapes. Hence we can,
- Find the current tape one symbol by scanning right from the start of the interleaved area until matching a 1, then observing the next symbol to the right.
- Advance the tape one head by scanning right from the start of the interleaved area until matching a 1, over writing the 1 with a blank, moving four more steps rightwards, and writing a 1.
And likewise for tape 2, was well as writing symbols on the the head and moving the head one to the left.

__*We must maintain the data structure invariant for the interleaved area that there is exactly one 1 and one 2 in that area.*__

**The staging area: gather/execute.**

The staging area for a 2-tape machine is of format,

$$
(\,\_\,|\,r\,|\,l\,|\,n\,)(\,\sigma_1\,|\,1\,)(\,\_\,|\,r\,|\,l\,|\,n\,)(\,\sigma_2\,|\,2\,)
$$

The meaning depends on whether we are in a gather stage or execute stage. The gather stage will hunt for the symbols currently under the heads on tape one and two, and will copy them to $\sigma_1$ and $\sigma_2$. In the gather phase, the other tape contents of the staging area do not matter. The 1 and 2 in the staging area is just hackery to help simplify the code.

In the excute phase, the four cells of the staging area are set to the output of the transition funtion. The excute phase will copy from the staging area to the interleaved area the symbols, and update the head positions in the interleaved area.

__*This Notebook shows the gather code. The excute code is left as an exercise.*__


**The soft-state area: encoding and use of the transition table.**

TBA (this is complicated).

**Putting it together**

So simulation repeats the three phases:

1. Gather the current symbols from the interleaved area into the staging area.
2. Run the state transition tree to write into the state area and staging area.
3. Execute the transition by copying and updating from the staging area into the interleaved area.




### Main Classes

In [16]:
import string
import sys
import os
import argparse
import re

#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 4 apr 2020
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#


#
# BETA VERSION .. RELOAD OFTEN 
#

class TuringMachine:

    def __init__(self,verbose="none",endmarker=False):
        self.start_state = ""
        self.accept_states = set()
        self.reject_states = set()
        self.transitions = {}
        self.current_state = ""
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.verbose_levels = {"none":0, "verbose":1, "debug":2}
        self.tape = [' ']
        self.position = 0
        self.verbose = self.verbose_levels[verbose]
        self.endmarker = endmarker

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        # change '_' to ' '
        self.tape = [' ' if symbol=='_' else symbol 
                         for symbol in tape_string]
        if self.endmarker:
            self.tape.insert(0,':')
        
    def set_verbose(self,verbose):
        self.verbose = 0
        if verbose in self.verbose_levels:
            self.verbose = self.verbose_levels[verbose]

    def set_endmarker(self,endmarker):
        self.endmarker = endmarker

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):
        """
        Returns None on success; else return an error string.
        """
        
        if self.verbose >= self.verbose_levels['debug']:
            print("adding transition:", 
                  state_from, read_symbol, write_symbol, action, state_to )

        if read_symbol =='_': 
            read_symbol = ' '
        if write_symbol =='_':
            write_symbol = ' '

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action, skipping."
        x = (state_from, read_symbol)
        if x in self.transitions:
            print("WARNING: multple outgoing states, skipping",x)
            return "WARNING: multiple outgoing states, skipping."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = ' '
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        """
        take one state transition, based on tape, states, and transitions.
        Returns None if ok; else returns unmatched transition
        """
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        else:
            if self.verbose>=self.verbose_levels['debug']:
                print('current state:', c_s, 'current symbol: |', 
                      self.tape[self.position],'| current position: ', self.position)
            return str(x)
        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = ' '
        if action == 'n':
            pass
   
        if shout:
            self.print_tape()

        if self.verbose >= self.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
        self.step_counter += 1
        return None

    def compute_tm(self,tape_string,step_limit=0):
        self.restart(tape_string)
        step = 0

        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if res:
                return ("no transition",res)
            step += 1
            if step > step_limit:
                return ("step limit",step,self.str_tape())
            if self.verbose >= self.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        cause = "reject"
        if self.current_state in self.accept_states:
            cause = "accept"
        the_tape = self.str_tape()
        return (cause,the_tape)

    def str_tape(self):
        t, p = self.tape, self.position
        return ''.join(['|'] + t[:p] + ['>'] + [t[p]] + t[p+1:] + ['|'])
        
    def print_tape(self):
        s = self.str_tape()
        print(''.join(["state:\t"]+[self.current_state]+["\ntape:\t"]+[s]))
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
        # print("tape:\n\t",self.tape)
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|:)\s+(\w|:)\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res)
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                break

            line_no += 1
        return

### end class MachineParser



### Helper definitions

In [17]:

def create_and_test_turing_machine(tm_description, test_cases, 
                                   verbose='none', endmarker=False):
    tm = TuringMachine(verbose,endmarker)
    MachineParser.turing(tm,tm_description)
    
    #print("\n\n*** THE TURING MACHINE ***")
    #if verbose!='none': tm.print_tm()

    print("\n\n*** TEST RUNS ***\n\n")

    for s in test_cases:
        tm.restart(s)
        s_fmt = tm.str_tape()
        print(''.join(["input:\t"]+[s_fmt]))
        # assume complexity is some quadratic
        t = tm.compute_tm(s,step_limit=10*(len(s)+5)**2)
        print("".join([t[0]]+[':\t']+[t[1]]))

    print("\n\n*** RUN COMPLETE ***\n\n")

def create_and_iterate_turing_machine(tm_description, starting_tape, count, 
                                   verbose='none', endmarker=False):
    
    tm = TuringMachine(verbose,endmarker)
    MachineParser.turing(tm,tm_description)
    
    #print("\n\n*** THE TURING MACHINE ***")
    #tm.print_tm()

    print("\n\n*** COUNT RUNS ***\n\n")

    tape = starting_tape
    for i in range(count):
        # assume complexity is some quadratic
        tape = tm.compute_tm(tape,step_limit=10*(len(tape)+5)**2)
        tape = tape[1][1:]

    print("\n\n*** RUN COMPLETE ***\n\n")

    
def combine_two_tapes(tape1,h1,tape2,h2):

    tape1 += '  '
    tape2 += '  '
    tape1 += ' '*len(tape2)
    tape2 += ' '*len(tape1)
    if (h1>=len(tape1)) :
         h1 = 0
    if (h2>=len(tape2)) :
        h2 = 0
        
    tape = ":    "
    tape_pos = 0
    for t1s, t2s in zip(tape1,tape2):
        head1 = ' '
        if tape_pos == h1:
            head1 = '1'
        head2 = ' '
        if tape_pos == h2:
            head2 = '2'
        tape += (head1+t1s+head2+t2s)
        tape_pos += 1

    return tape


def make_test_tape(sym1,act1,sym2,act2,tape1,h1,tape2,h2):
    t = combine_two_tapes(tape1,h1,tape2,h2)
    t = ''.join([':{}{}{}{}'.format(act1,sym1,act2,sym2),t[5:]])
    return t


### The Gather Program

In [18]:
mt1="""# Simulating k-tapes a on 1-tape TM
# for this example, reduce to 2 tapes, with tape alphabet a and b.
#
# the full tape has three sections.
# (1) a fixed length string recording the state of the machine under simulation
# (2) a staging area into which are copied the current tape symbols, to simulate
#     the transition, and into which are copied the the output of the transition, 
#     before the symbol and action are recorded in the tape area
# (3) the tape area, in which the two tapes are interleaved, and a place is left
#     to mark "here!" for each of the heads.
#
#   :{0|1|_}^n:{r|l|n|_}{a|b|_}{r|l|n|_}{a|b|_}({_|1}{a|b}{_|2}{a|b})+
#
# there must be exactly one 1 and exactly one 2 on the tape (the head must be somewhere)
#

#
# we start simpler though, 
#
#   :_{a|b|_|1}_{a|b|_|2}({_|1}{a|b}{_|2}{a|b})+
#
# that is - no soft_state area, only action allowed is _; and the 1 and 2 
# in the staging area are temporary markers. 

start: s
accept: a
reject: r

# gather_to_staging, execute_from_staging

# the ideas in gathering: setup the tape first so that the staging area is
#    _1_2
# then search rightwards for the first 2 (beyond the staging area).
# remember the symbol to the immediate right of the 2, and search 
# leftwards for the first 2 (it will be the 2 in the staging area).
# HACKER ALERT: I temporarily change the 2 in the tape area to an x, then
# replace it back to a 2.

# when the 2 in the staging are is found I replace it with the remembered symbol.
# then repeat the same for tape 1

# assert: at left tape end; 
state: s
    : : r gather_to_staging

# prepare the staging area with navigation markers
# assert: over leftmost symbol in staging area
state: gather_to_staging
    _ _ r mark_staging_1
state: mark_staging_1
    a 1 r mark_staging_2
    b 1 r mark_staging_2
    _ 1 r mark_staging_2
state: mark_staging_2
    _ _ r mark_staging_3
state: mark_staging_3
    a 2 r find_head_t2
    b 2 r find_head_t2
    _ 2 r find_head_t2
    
# assert: the staging area has been marked, 
#   the head is over the first symbol in the tape area
# goal: to find the tape 2 head
state: find_head_t2
    _ _ r find_head_t2
    a a r find_head_t2
    b b r find_head_t2
    1 1 r find_head_t2
    2 x r head_t2_found  # marking the head marker

state: head_t2_found
    a a l got_a_t2
    b b l got_b_t2
    _ _ l got___t2

state: got_a_t2
    x 2 l got_a_t2   # this is run once to restore the head marker
    _ _ l got_a_t2
    a a l got_a_t2
    b b l got_a_t2
    1 1 l got_a_t2
    2 a l find_head_t1
state: got_b_t2
    x 2 l got_b_t2   # this is run once to restore the head marker
    _ _ l got_b_t2
    a a l got_b_t2
    b b l got_b_t2
    1 1 l got_b_t2
    2 b l find_head_t1
state: got___t2
    x 2 l got___t2  # this is run once to restore the head marker
    _ _ l got___t2
    a a l got___t2
    b b l got___t2
    1 1 l got___t2
    2 _ l find_head_t1

# assert: tape 2 symbol gathered into staging area
# goal: gather tape 1 symbol into staging area

state: find_head_t1
    _ _ r find_head_t1
    a a r find_head_t1
    b b r find_head_t1
    2 2 r find_head_t1
    1 x r head_t1_found

state: head_t1_found
    a a l got_a_t1
    b b l got_b_t1
    _ _ l got___t1

state: got_a_t1
    x 1 l got_a_t1
    _ _ l got_a_t1
    a a l got_a_t1
    b b l got_a_t1
    1 a l gather_done
    2 2 l got_a_t1
state: got_b_t1
    x 1 l got_b_t1
    _ _ l got_b_t1
    a a l got_b_t1
    b b l got_b_t1
    1 b l gather_done
    2 2 l got_b_t1
state: got___t1
    x 1 l got___t1
    _ _ l got___t1
    a a l got___t1
    b b l got___t1
    1 _ l gather_done
    2 2 l got___t1

state: gather_done
    a a l gather_done
    b b l gather_done
    _ _ l gather_done
    1 1 l gather_done
    2 2 l gather_done
    : : n a

"""


In [19]:
# 
# testing the gather TM program
#

tape1 = "aaa"
tape2 = "bbbbb"
tt = []
for i in range(len(tape1)+1):
    for j in range(len(tape2)+1):
        tt += [combine_two_tapes("aaa",i,"bbbbb",j)]

create_and_test_turing_machine(mt1,tt)




*** TEST RUNS ***


input:	|>:    1a2b a b a b   b   b                            |
accept:	|>: a b1a2b a b a b   b   b                            |
input:	|>:    1a b a2b a b   b   b                            |
accept:	|>: a b1a b a2b a b   b   b                            |
input:	|>:    1a b a b a2b   b   b                            |
accept:	|>: a b1a b a b a2b   b   b                            |
input:	|>:    1a b a b a b  2b   b                            |
accept:	|>: a b1a b a b a b  2b   b                            |
input:	|>:    1a b a b a b   b  2b                            |
accept:	|>: a b1a b a b a b   b  2b                            |
input:	|>:    1a b a b a b   b   b  2                         |
accept:	|>: a  1a b a b a b   b   b  2                         |
input:	|>:     a2b1a b a b   b   b                            |
accept:	|>: a b a2b1a b a b   b   b                            |
input:	|>:     a b1a2b a b   b   b                            |
accept:	|>:

### Hint for the write-down problem (aka execute)

In [23]:
mt_writedown="""# hints to the write-down problem set (aka execute)

start: s
accept: a
reject: r

# the first 5 cells of the tape are 
#     :(r|l|n)(a|b|_)(r|l|n)(a|b|_)
# beginning in the 6th cell are repeating sequence of 4 cells, each like
#     (_|1)(a|b|_)(_|2)(a|b|_)

# data structure invariant:
# - there are exactly one 1 and one 2 on the tape. they are the tape 1 and tape 2 headmarkers

# mission: to accomplish the work asked for in the 4 cells to the right of the 
# end of tape marker
# - to write an a, b or _ just one beyond head marker 1
# - to write an a, b or _ just one beyond head marker 2
# - to move left or right head marker 1, or leave it in place
# - to move left or right head marker 2, or leave it in place

# assume at the start state you are over the left most tape cell, seeing a :

# this takes case of the first of the 4 missions, write-down of a tape 1 symbol

state: s
    : : R get_write_1

# mission: to identify a, b or _ as the writedown symbol
state: get_write_1
    r r r get_write_1 # just skip ...
    l l r get_write_1 # ... over the  ...
    n n r get_write_1 # ... action code
    a : R write_1_a   # make a phoney left end of tape at vacated symbol
    b : R write_1_b
    _ : R write_1__
 
# mission: to writedown an a
state: write_1_a
    _ _ r write_1_a  # travel right until ...
    r r r write_1_a
    l l r write_1_a
    n n r write_1_a
    a a r write_1_a
    b b r write_1_a
    2 2 r write_1_a
    1 1 R found_1_a # the number 1 is found
    
state: found_1_a
    a a L leot_1
    b a L leot_1
    _ a L leot_1

# the next bunch of lines does the same as writedown of an a,
# but one bunch delivers the b, the other the _.
# the code flow then merges again into leot_1

state: write_1_b
    _ _ r write_1_b  # travel right until ...
    r r r write_1_b
    l l r write_1_b
    n n r write_1_b
    a a r write_1_b
    b b r write_1_b
    2 2 r write_1_b
    1 1 R found_1_b # the number 1 is found
    
state: found_1_b
    a b L leot_1
    b b L leot_1
    _ b L leot_1

state: write_1__
    _ _ r write_1__  # travel right until ...
    r r r write_1__
    l l r write_1__
    n n r write_1__
    a a r write_1__
    b b r write_1__
    2 2 r write_1__
    1 1 R found_1__ # the number 1 is found
    
state: found_1__
    a _ L leot_1
    b _ L leot_1
    _ _ L leot_1

# here the code flow merges; the a, b or _ has been written under the 
# head marker of tape 1; head on back to the : symbol.

state: leot_1
    a a l leot_1
    b b l leot_1
    _ _ l leot_1
    1 1 l leot_1
    2 2 l leot_1
    n n l leot_1
    r r l leot_1
    l l l leot_1
    : : N write_1_done   

# reminder: at the point the first 5 cells of the tape look like
#    :(r|l|n):(r|l|n)(a|b|_)
 
# now go on with the rest of the write-down
state: write_1_done 
    : : n a
    
"""

    
    
tape1 = "aaaaaa"
tape2 = "bbbbbbbbb"
tape = make_test_tape('b','n','a',"n",tape1,0,tape2,0)
create_and_test_turing_machine(mt_writedown,[tape])
tape = make_test_tape('b','n','a',"n",tape1,3,tape2,2)
create_and_test_turing_machine(mt_writedown,[tape])
tape = make_test_tape('b','n','a',"n",tape1,3,tape2,6)
create_and_test_turing_machine(mt_writedown,[tape])





*** TEST RUNS ***


input:	|>:nbna1a2b a b a b a b a b a b   b   b   b                                        |
state:	get_write_1
tape:	|:>nbna1a2b a b a b a b a b a b   b   b   b                                        |
state:	write_1_b
tape:	|:n:>na1a2b a b a b a b a b a b   b   b   b                                        |
state:	found_1_b
tape:	|:n:na1>a2b a b a b a b a b a b   b   b   b                                        |
state:	leot_1
tape:	|:n:na>1b2b a b a b a b a b a b   b   b   b                                        |
state:	write_1_done
tape:	|:n>:na1b2b a b a b a b a b a b   b   b   b                                        |
accept:	|:n>:na1b2b a b a b a b a b a b   b   b   b                                        |


*** RUN COMPLETE ***




*** TEST RUNS ***


input:	|>:nbna a b a b a2b1a b a b a b   b   b   b                                        |
state:	get_write_1
tape:	|:>nbna a b a b a2b1a b a b a b   b   b   b                                        |
st